In [ ]:
import cv2
import mediapipe as mp

# Initialize the face detection model from Mediapipe
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

# Open the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam was opened successfully
if not cap.isOpened():
    print("Error: Couldn't open the cam")
    exit(1)

# Inform the user how to exit the program
print("Press 'q' to exit")

# Start a loop to continuously capture video frames
while True:
    sucsess, frame = cap.read()  # Capture a frame from the webcam
    if not sucsess:
        break

    # Convert the frame from BGR to RGB color format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame for face detection using the Mediapipe face detection model
    results = face_detection.process(frame_rgb)

    # Check if any faces were detected
    if results.detections:
        for detection in results.detections:
            # Extract the bounding box coordinates of the detected face
            bboxC = detection.location_data.relative_bounding_box
            h, w, _ = frame.shape

            # Calculate the x, y, width, and height of the bounding box in pixel values
            x, y, width, height = (int(bboxC.xmin * w), int(bboxC.ymin * h),
                                   int(bboxC.width * w), int(bboxC.height * h))

            # Ensure the coordinates do not go out of frame bounds
            x, y = max(0, x), max(0, y)
            width, height = min(w - x, width), min(h - y, height)

            # Extract the region of the frame corresponding to the detected face
            face_region = frame[y:y+height, x:x+width]

            # Apply a Gaussian blur to the face region
            blurred_face = cv2.GaussianBlur(face_region, (55, 55), 10)

            # Replace the original face region in the frame with the blurred version
            frame[y:y+height, x:x+width] = blurred_face

    # Display the processed frame with the blurred face
    cv2.imshow("Face Blur Webcam(MediaPipe)", frame)

    # Check if the user pressed the 'q' key to exit the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the video capture object and close the OpenCV windows
cap.release()
cv2.destroyAllWindows()
